In [133]:
# code by Peter Solis
# import dependencies
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [105]:
# read in data
df = pd.read_csv('../01 - Data Crunching/dataset.csv')

In [106]:
# translating our target into categorical values for these models to work with
# https://www.codementor.io/@agarrahul01/multiclass-classification-using-random-forest-on-scikit-learn-library-hkk4lwawu was referenced for how to make a tree / forest model with these in mind
target_names = pd.factorize(df['Target'])[1]
df['Target'] = pd.factorize(df['Target'])[0]
for i in range(len(definitions)):
    print(f'{i} - {definitions[i]}')

0 - Dropout
1 - Graduate
2 - Enrolled


In [107]:
# split based on data type
categorical_df = df.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18]].copy()
numerical_df = df.iloc[:,[2,17,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33]].copy()
y = df[['Target']].copy()

In [108]:
# make non-binary categorical columns clearly categorical - you may want to also bin these in some cases
# more info on what each category means can be found here:
# https://www.mdpi.com/2306-5729/7/11/146
def categorize(cell):
    return f'cat_{cell}'
col_to_adjust = ['Marital status',
                 'Application mode',
                 'Course',
                 'Previous qualification',
                 'Nacionality',
                 "Mother's qualification",
                 "Father's qualification",
                 "Mother's occupation",
                 "Father's occupation"]
for col in col_to_adjust:
    categorical_df[col] = categorical_df[col].apply(categorize)
categorical_df.head()

,Marital status,Application mode,Course,Daytime/evening attendance,Previous qualification,Nacionality,Mother's qualification,Father's qualification,Mother's occupation,Father's occupation,Displaced,Educational special needs,Debtor,Tuition fees up to date,Gender,Scholarship holder,International
0,cat_1,cat_8,cat_2,1,cat_1,cat_1,cat_13,cat_10,cat_6,cat_10,1,0,0,1,1,0,0
1,cat_1,cat_6,cat_11,1,cat_1,cat_1,cat_1,cat_3,cat_4,cat_4,1,0,0,0,1,0,0
2,cat_1,cat_1,cat_5,1,cat_1,cat_1,cat_22,cat_27,cat_10,cat_10,1,0,0,0,1,0,0
3,cat_1,cat_8,cat_15,1,cat_1,cat_1,cat_23,cat_27,cat_6,cat_4,1,0,0,1,0,0,0
4,cat_2,cat_12,cat_3,0,cat_1,cat_1,cat_22,cat_28,cat_10,cat_10,0,0,0,1,0,0,0


In [109]:
# finish pre-processing categorical data
# I referenced the tables of most important variables found in the cell above, and decided to exclude some categorical data that didn't play a huge role and would greatly increase the number of columns
# important categorical data with tons of values
to_drop = ["Application mode",
           "Nacionality",
           "Mother's qualification",
           "Father's qualification"]
cat_df_2 = pd.get_dummies(categorical_df.drop(columns = to_drop, axis = 1))
# try using PCA to deal with the huge number of values, we'll give it twice the number of columns of the actual values
from sklearn.decomposition import PCA
pca = PCA(n_components = 10)
pca_cat_df = pd.DataFrame(pca.fit_transform(cat_df_2), columns = ['PCA1','PCA2','PCA3','PCA4','PCA5','PCA6','PCA7','PCA8','PCA9','PCA10'])

In [110]:
# these are the categorical columns that stay as one column
others = ['Daytime/evening attendance',
          'Displaced',
          'Educational special needs',
          'Debtor',
          'Tuition fees up to date',
          'Gender',
          'Scholarship holder',
          'International']
parsed_cat_df = pd.concat([pca_cat_df, categorical_df[others]], axis = 1)
parsed_cat_df

,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,Daytime/evening attendance,Displaced,Educational special needs,Debtor,Tuition fees up to date,Gender,Scholarship holder,International
0,-0.162154,0.028001,-0.871983,-0.003359,-0.282293,-0.505528,-0.168626,-0.307884,0.555932,-0.053045,1,1,0,0,1,1,0,0
1,-0.230441,-0.508816,-0.888290,-0.125946,-0.399207,0.014567,-0.085585,-0.039866,-0.098055,0.019764,1,1,0,0,0,1,0,0
2,0.442845,0.679988,-1.137406,-0.198112,-0.015490,0.026529,-0.076910,-0.163479,0.025311,0.335386,1,1,0,0,0,1,0,0
3,-0.720104,-0.190389,-0.002909,-0.298947,-0.633589,-0.212604,-0.349174,-0.193115,0.115808,-0.138063,1,1,0,0,1,0,0,0
4,1.459907,0.612460,0.796143,-0.487261,-0.180097,-0.687637,0.329710,-0.474647,-0.070922,0.050034,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4419,0.111358,-0.748511,-0.315049,0.578175,-0.166301,-0.397680,-0.377930,-0.099675,0.045516,0.018740,1,0,0,0,1,1,0,0
4420,0.261952,0.938418,-0.512631,-0.692239,-0.074014,0.537430,-0.635663,-0.259445,-0.222414,0.828014,1,1,0,1,0,0,0,1
4421,-0.288204,1.448425,0.096068,0.118647,0.315143,-0.259998,0.161980,0.441843,0.364453,0.168819,1,1,0,0,1,0,1,0
4422,-0.756696,0.173572,0.243609,0.278156,0.066636,0.162239,0.132819,-0.386641,0.084041,-0.137369,1,1,0,0,1,0,1,0


In [111]:
# finish preprocessing by scaling and doing PCA
scaled_num_df = pd.DataFrame(StandardScaler().fit_transform(numerical_df), columns = numerical_df.columns)
final_df = pd.concat([parsed_cat_df, scaled_num_df], axis = 1)
final_df.head()

,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,-0.162154,0.028001,-0.871983,-0.003359,-0.282293,-0.505528,-0.168626,-0.307884,0.555932,-0.053045,...,-0.199273,-0.282442,-2.838337,-2.042630,-1.471527,-1.963489,-0.199441,-0.287638,0.124386,0.765761
1,-0.230441,-0.508816,-0.888290,-0.125946,-0.399207,0.014567,-0.085585,-0.039866,-0.098055,0.019764,...,-0.199273,-0.282442,-0.105726,-0.522682,0.518904,0.659562,-0.199441,0.876222,-1.105222,0.347199
2,0.442845,0.679988,-1.137406,-0.198112,-0.015490,0.026529,-0.076910,-0.163479,0.025311,0.335386,...,-0.199273,-0.282442,-0.105726,-2.042630,-1.471527,-1.963489,-0.199441,-0.287638,0.124386,0.765761
3,-0.720104,-0.190389,-0.002909,-0.298947,-0.633589,-0.212604,-0.349174,-0.193115,0.115808,-0.138063,...,-0.199273,-0.282442,-0.105726,0.490616,0.187165,0.416450,-0.199441,-0.813253,-1.466871,-1.375511
4,1.459907,0.612460,0.796143,-0.487261,-0.180097,-0.687637,0.329710,-0.474647,-0.070922,0.050034,...,-0.199273,-0.282442,-0.105726,-0.522682,0.518904,0.531608,-0.199441,0.876222,-1.105222,0.347199


In [117]:
# just testing w/ PCA again after all variables in
pca2 = PCA(n_components = 10)
pca2_df = pd.DataFrame(pca2.fit_transform(final_df))
sum(pca2.explained_variance_ratio_)

0.8241355822711064

In [118]:
# split data
X_train, X_test, y_train, y_test = train_test_split(final_df, y, random_state = 0)
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(pca2_df, y, random_state = 0)

In [119]:
# first model, tree
tree_model = DecisionTreeClassifier()
tree_model.fit(X_train, y_train)
tree_predict = tree_model.predict(X_test)

In [120]:
# results
print(classification_report(y_test, tree_predict, target_names = target_names))
confusion_matrix(y_test, tree_predict)

              precision    recall  f1-score   support

     Dropout       0.73      0.72      0.72       353
    Graduate       0.80      0.78      0.79       560
    Enrolled       0.36      0.39      0.38       193

    accuracy                           0.69      1106
   macro avg       0.63      0.63      0.63      1106
weighted avg       0.70      0.69      0.69      1106



array([[253,  45,  55],
       [ 44, 435,  81],
       [ 51,  66,  76]], dtype=int64)

In [122]:
# second model, tree w/ fully PCA data
tree_model_pca = DecisionTreeClassifier()
tree_model_pca.fit(X_train_pca, y_train_pca)
tree_predict_pca = tree_model_pca.predict(X_test_pca)

In [123]:
# results
print(classification_report(y_test_pca, tree_predict_pca, target_names = target_names))
confusion_matrix(y_test_pca, tree_predict_pca)

              precision    recall  f1-score   support

     Dropout       0.65      0.68      0.66       353
    Graduate       0.80      0.75      0.77       560
    Enrolled       0.33      0.36      0.34       193

    accuracy                           0.66      1106
   macro avg       0.59      0.59      0.59      1106
weighted avg       0.67      0.66      0.66      1106



array([[239,  49,  65],
       [ 65, 418,  77],
       [ 66,  58,  69]], dtype=int64)

In [127]:
# third model, forest
forest_model = RandomForestClassifier()
forest_model.fit(X_train, y_train['Target'].ravel())
forest_predict = forest_model.predict(X_test)

In [128]:
# results
print(classification_report(y_test, forest_predict, target_names = target_names))
confusion_matrix(y_test, forest_predict)

              precision    recall  f1-score   support

     Dropout       0.79      0.77      0.78       353
    Graduate       0.80      0.95      0.87       560
    Enrolled       0.56      0.27      0.37       193

    accuracy                           0.78      1106
   macro avg       0.72      0.67      0.67      1106
weighted avg       0.76      0.78      0.75      1106



array([[273,  48,  32],
       [ 17, 534,   9],
       [ 57,  83,  53]], dtype=int64)

In [129]:
# fourth model, forest w/ fully PCA data
forest_model_pca = RandomForestClassifier()
forest_model_pca.fit(X_train_pca, y_train_pca['Target'].ravel())
forest_predict_pca = forest_model_pca.predict(X_test_pca)

In [130]:
# results
print(classification_report(y_test_pca, forest_predict_pca, target_names = target_names))
confusion_matrix(y_test_pca, forest_predict_pca)

              precision    recall  f1-score   support

     Dropout       0.76      0.73      0.75       353
    Graduate       0.80      0.93      0.86       560
    Enrolled       0.53      0.33      0.40       193

    accuracy                           0.76      1106
   macro avg       0.70      0.66      0.67      1106
weighted avg       0.74      0.76      0.74      1106



array([[258,  60,  35],
       [ 20, 518,  22],
       [ 61,  69,  63]], dtype=int64)

In [140]:
# fifth model, forest with more estimators
forest_model_250 = RandomForestClassifier(n_estimators = 250)
forest_model_250.fit(X_train, y_train['Target'].ravel())
forest_predict_250 = forest_model_250.predict(X_test)

In [141]:
# results
print(classification_report(y_test, forest_predict_250, target_names = target_names))
confusion_matrix(y_test, forest_predict_250)

              precision    recall  f1-score   support

     Dropout       0.80      0.78      0.79       353
    Graduate       0.81      0.95      0.87       560
    Enrolled       0.55      0.30      0.39       193

    accuracy                           0.78      1106
   macro avg       0.72      0.67      0.68      1106
weighted avg       0.76      0.78      0.76      1106



array([[274,  46,  33],
       [ 15, 530,  15],
       [ 54,  81,  58]], dtype=int64)

In [152]:
# sixth model, Support Vector Machine
sv_model = SVC(kernel = 'linear')
sv_model.fit(X_train, y_train['Target'].ravel())
sv_predict = sv_model.predict(X_test)

In [153]:
# results
print(classification_report(y_test, sv_predict, target_names = target_names))
confusion_matrix(y_test, sv_predict)

              precision    recall  f1-score   support

     Dropout       0.82      0.71      0.76       353
    Graduate       0.80      0.94      0.87       560
    Enrolled       0.53      0.39      0.45       193

    accuracy                           0.77      1106
   macro avg       0.72      0.68      0.69      1106
weighted avg       0.76      0.77      0.76      1106



array([[250,  51,  52],
       [ 17, 527,  16],
       [ 38,  79,  76]], dtype=int64)

In [154]:
# seventh model, Support Vector Machine w/ fully PCA data
sv_model_pca = SVC(kernel = 'linear')
sv_model_pca.fit(X_train_pca, y_train_pca['Target'].ravel())
sv_predict_pca = sv_model_pca.predict(X_test_pca)

In [155]:
# results
print(classification_report(y_test_pca, sv_predict_pca, target_names = target_names))
confusion_matrix(y_test_pca, sv_predict_pca)

              precision    recall  f1-score   support

     Dropout       0.77      0.72      0.74       353
    Graduate       0.78      0.92      0.84       560
    Enrolled       0.52      0.32      0.39       193

    accuracy                           0.75      1106
   macro avg       0.69      0.65      0.66      1106
weighted avg       0.73      0.75      0.73      1106



array([[253,  62,  38],
       [ 29, 513,  18],
       [ 48,  84,  61]], dtype=int64)